## Memo

I created this notebook to confirm @swaralipibose 's [notebook](https://www.kaggle.com/swaralipibose/new-feature-score-improved)

I only clustered pivotted u_in by KMeans.

## Import

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
from sklearn.cluster import KMeans

In [ ]:
train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
test = pd.read_csv("../input/ventilator-pressure-prediction/test.csv")

In [ ]:
train["count"] = train["id"] % 80
train["count"] = train["count"].replace({0: 80})
train_pivot = train.pivot(index="count", columns="breath_id", values="u_in")

test["count"] = test["id"] % 80
test["count"] = test["count"].replace({0: 80})
test_pivot = test.pivot(index="count", columns="breath_id", values="u_in")

In [ ]:
kmeans = KMeans(3, random_state=0)
kmeans.fit(train_pivot.T)
k_u_in_train = kmeans.predict(train_pivot.T)
k_u_in_test = kmeans.predict(test_pivot.T)

In [ ]:
train["cluster"] = np.repeat(k_u_in_train, 80)
test["cluster"] = np.repeat(k_u_in_test, 80)

## plot cluster

In [ ]:
# median u_in in each cluster
plt.plot(train.query("cluster == 0").groupby("count")["u_in"].median().values, label="Cluster0")
plt.plot(train.query("cluster == 1").groupby("count")["u_in"].median().values, label="Cluster1")
plt.plot(train.query("cluster == 2").groupby("count")["u_in"].median().values, label="Cluster2")

In [ ]:
# mean u_in in each cluster
plt.plot(train.query("cluster == 0").groupby("count")["u_in"].mean().values, label="Cluster0")
plt.plot(train.query("cluster == 1").groupby("count")["u_in"].mean().values, label="Cluster1")
plt.plot(train.query("cluster == 2").groupby("count")["u_in"].mean().values, label="Cluster2")
plt.legend()

In [ ]:
train_cluster_dummied = pd.get_dummies(train["cluster"])
train_cluster_dummied.columns = ["Cluster1", "Cluster2", "Cluster3"]
test_cluster_dummied = pd.get_dummies(test["cluster"])
test_cluster_dummied.columns = ["Cluster1", "Cluster2", "Cluster3"]

In [ ]:
train_cluster_dummied.head()

In [ ]:
train_cluster_dummied.to_csv("train.cluster.csv")
test_cluster_dummied.to_csv("test.cluster.csv")